# CellCharter

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 11.03.2024
- **Date of Last Modification:** 18.07.2024

- The CellCharter source code is available at https://github.com/CSOgroup/cellcharter.
- The corresponding publication is "Varrone, M., Tavernari, D., Santamaria-Martínez, A., Walsh, L. A. & Ciriello, G. CellCharter reveals spatial cell niches associated with tissue remodeling and cell plasticity. Nat. Genet. 56, 74–84 (2024)".
- The workflow of this notebook follows the tutorial from https://cellcharter.readthedocs.io/en/latest/notebooks/cosmx_human_nsclc.html.

## 1. Setup

### 1.1 Import Libraries

In [ ]:
import gc
import os
import time
from datetime import datetime

import anndata as ad
import squidpy as sq
from spatialdata_plot.pl.utils import _get_colors_for_categorical_obs # this import is crucial, otherwise core gets dumped
import cellcharter as cc
import pandas as pd
import scanpy as sc
import scvi
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp

### 1.2 Define Parameters

In [ ]:
model_name = "cellcharter"
latent_key = f"{model_name}_latent"
mapping_entity_key = "reference"
condition_key = "batch"
counts_key = "counts"
spatial_key = "spatial"
adj_key = "spatial_connectivities"

### 1.3 Run Notebook Setup

In [ ]:
sc.set_figure_params(figsize=(6, 6))

In [ ]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [ ]:
st_data_gold_folder_path = "../../../datasets/st_data/gold"
st_data_results_folder_path = "../../../datasets/st_data/results" 
figure_folder_path = f"../../../figures"
benchmarking_folder_path = "../../../artifacts/sample_integration_method_benchmarking"

# Create required directories
os.makedirs(st_data_gold_folder_path, exist_ok=True)
os.makedirs(st_data_results_folder_path, exist_ok=True)

## 2. CellCharter Model

### 2.1 Define Training Function

In [ ]:
def train_cellcharter_models(dataset,
                             reference_batches,
                             cell_type_key,
                             adata_new=None,
                             n_start_run=1,
                             n_end_run=8,
                             n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16],
                             plot_latent_umaps=False,
                             filter_genes: bool=False,
                             n_svg: int=3000):    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:  
        adata_batch_list = []
        if reference_batches is not None:
            for batch in reference_batches:
                adata_batch = ad.read_h5ad(
                    f"{st_data_gold_folder_path}/{dataset}_{batch}.h5ad")
                adata_batch.obs[mapping_entity_key] = "reference"
                adata_batch_list.append(adata_batch)
            adata_original = ad.concat(adata_batch_list, join="inner")
        else:
            adata_original = ad.read_h5ad(f"{st_data_gold_folder_path}/{dataset}.h5ad")

        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        adata_new.obs[condition_key] = adata_original.obs[condition_key]
        adata_new.obs[mapping_entity_key] = adata_original.obs[mapping_entity_key] 
        del(adata_original)
        
    model_seeds = list(range(10))
    for run_number, n_neighbors in zip(np.arange(n_start_run, n_end_run+1), n_neighbor_list):
        # n_neighbors is here only used for the latent neighbor graph construction used for
        # UMAP generation and clustering as scVI is not a spatial method

        # Load trained SCVI results
        adata = sc.read_h5ad(f"{benchmarking_folder_path}/{dataset}_scvi.h5ad")
        
        start_time = time.time()

        adata.obsm["latent_scvi"] = adata.obsm[f"scvi_latent_run{run_number}"].astype(np.float32)

        # Compute spatial neighborhood graph
        sq.gr.spatial_neighbors(adata,
                                library_key=condition_key,
                                coord_type="generic",
                                spatial_key="spatial",
                                n_neighs=n_neighbors)

        # Use CellCharter as per tutorial
        cc.gr.remove_long_links(adata)
        cc.gr.aggregate_neighbors(adata, n_layers=3, use_rep="latent_scvi", out_key=latent_key,  sample_key=condition_key)
        
        # Measure time for model training
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Duration of model training in run {run_number}: "
              f"{int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds.")
        adata_new.uns[f"{model_name}_model_training_duration_run{run_number}"] = (
            adata.uns[f"scvi_model_training_duration_run{run_number}"] + elapsed_time)
        
        # Store latent representation
        adata_new.obsm[latent_key + f"_run{run_number}"] = adata.obsm[latent_key]
        
        # Use latent representation for UMAP generation
        sc.pp.neighbors(adata_new,
                        use_rep=f"{latent_key}_run{run_number}",
                        key_added=f"{latent_key}_run{run_number}")
        sc.tl.umap(adata_new,
                   neighbors_key=f"{latent_key}_run{run_number}")
        adata_new.obsm[f"{latent_key}_run{run_number}_X_umap"] = adata_new.obsm["X_umap"]
        del(adata_new.obsm["X_umap"])

        # Store intermediate adata to disk
        adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad")  

        # Free memory
        del(adata)
        gc.collect()
        
    # Store final adata to disk
    adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad") 

### 2.2 Train Models on Benchmarking Datasets

In [ ]:
train_cellcharter_models(dataset="seqfish_mouse_organogenesis",
                         reference_batches=[f"batch{i}" for i in range(1,7)],
                         cell_type_key="celltype_mapped_refined",
                         adata_new=None,
                         n_start_run=1,
                         n_end_run=8,
                         n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_cellcharter_models(dataset=f"seqfish_mouse_organogenesis_subsample_{subsample_pct}pct",
                             reference_batches=[f"batch{i}" for i in range(1,7)],
                             cell_type_key="celltype_mapped_refined",
                             adata_new=None,
                             n_start_run=1,
                             n_end_run=8,
                             n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_cellcharter_models(dataset="seqfish_mouse_organogenesis_imputed",
                         reference_batches=[f"batch{i}" for i in range(1,7)],
                         cell_type_key="celltype_mapped_refined",
                         adata_new=None,
                         n_start_run=1,
                         n_end_run=8,
                         n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16],
                         filter_genes=True,
                         n_svg=3000)

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_cellcharter_models(dataset=f"seqfish_mouse_organogenesis_imputed_subsample_{subsample_pct}pct",
                             reference_batches=[f"batch{i}" for i in range(1,7)],
                             cell_type_key="celltype_mapped_refined",
                             adata_new=None,
                             n_start_run=1,
                             n_end_run=8,
                             n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16],
                             filter_genes=True,
                             n_svg=3000)

In [ ]:
train_cellcharter_models(dataset="nanostring_cosmx_human_nsclc",
                         reference_batches=[f"batch{i}" for i in range(1, 4)],
                         cell_type_key="cell_type",
                         adata_new=None,
                         n_start_run=1,
                         n_end_run=8,
                         n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]: # might be reversed in stored object
    train_cellcharter_models(dataset=f"nanostring_cosmx_human_nsclc_subsample_{subsample_pct}pct",
                             reference_batches=[f"batch{i}" for i in range(1,4)],
                             cell_type_key="cell_type",
                             adata_new=None,
                             n_start_run=1,
                             n_end_run=8,
                             n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])